In [1]:
import pandas as pd
import numpy as np

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import RandomizedSearchCV
from conlleval import evaluate

D:\NEGATIVE\ProgramFiles\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\NEGATIVE\ProgramFiles\anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
data = pd.read_csv("tes.csv", encoding="latin1")

In [3]:
data = data.fillna(method="ffill")

In [4]:
data.tail(10)

,Surat,Ayat,Sentence #,Kata,Label
36803,6.0,165.0,Sentence: 954,cepat,O
36804,6.0,165.0,Sentence: 954,siksaan-nya,O
36805,6.0,165.0,Sentence: 954,dan,O
36806,6.0,165.0,Sentence: 954,sesungguhnya,O
36807,6.0,165.0,Sentence: 954,dia,O
36808,6.0,165.0,Sentence: 954,maha,O
36809,6.0,165.0,Sentence: 954,pengampun,O
36810,6.0,165.0,Sentence: 954,lagi,O
36811,6.0,165.0,Sentence: 954,maha,O
36812,6.0,165.0,Sentence: 954,penyayang,O


In [5]:
words = list(set(data["Kata"].values))

In [6]:
n_words = len(words); n_words

3487

In [7]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Kata"].values.tolist(),
                                                     s["Label"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [8]:
getter = SentenceGetter(data)

In [9]:
sent = getter.get_next()

In [10]:
sentences = getter.sentences

In [11]:
# def word2features(sent, i):
#     word = sent[i][0]

#     features = {
#         'bias': 1.0,
#         'word.lower()': word.lower(),
#         'word[-3:]': word[-3:],
#         'word[-2:]': word[-2:],
#         'word.isupper()': word.isupper(),
#         'word.istitle()': word.istitle(),
#         'word.isdigit()': word.isdigit(),
#     }
#     if i > 0:
#         word1 = sent[i-1][0]
#         features.update({
#             '-1:word.lower()': word1.lower(),
#             '-1:word.istitle()': word1.istitle(),
#             '-1:word.isupper()': word1.isupper(),
#         })
#     else:
#         features['BOS'] = True

#     if i < len(sent)-1:
#         word1 = sent[i+1][0]
#         features.update({
#             '+1:word.lower()': word1.lower(),
#             '+1:word.istitle()': word1.istitle(),
#             '+1:word.isupper()': word1.isupper(),
#         })
#     else:
#         features['EOS'] = True

#     return features


# def sent2features(sent):
#     return [word2features(sent, i) for i in range(len(sent))]

# def sent2labels(sent):
#     return [label for token, label in sent]

# def sent2tokens(sent):
#     return [token for token, label in sent]

In [58]:
def word2features(sent, i):
        word = sent[i][1]
        #postag = sent[i][2]

        features = {
            'bias': 1.0, 
#             'word.lower()': word.lower(), 
            'word': word,
            'word[-3:]': word[-3:],
            'word[-2:]': word[-2:],
            'word[-1:]': word[-1:],
            'word[:3]': word[:3],
            'word[:2]': word[:2],
            'word[:1]': word[:1],
            
#             'word.isupper()': word.isupper(),
            'word.islower()': word.islower(),
            'word.istitle()': word.istitle(),
#             'word.isdigit()': word.isdigit(),
           # 'postag': postag,
            
#             'word_pls_1.lower()': '', 
            'word_pls_1': '',
            'word_pls_1[-3:]': '',
            'word_pls_1[-2:]': '',
            'word_pls_1[-1:]': '',
            'word_pls_1[:3]': '',
            'word_pls_1[:2]': '',
            'word_pls_1[:1]': '',
#             'word_pls_1.isupper()': False,
            'word_pls_1.islower()': False,
            'word_pls_1.istitle()': False,
#             'word_pls_1.isdigit()': False,
           # 'postag_pls_1': '',
            
#             'word_min_1.lower()': '', 
            'word_min_1': '',
            'word_min_1[-3:]': '',
            'word_min_1[-2:]': '',
            'word_min_1[-1:]': '',
            'word_min_1[:3]': '',
            'word_min_1[:2]': '',
            'word_min_1[:1]': '',
#             'word_min_1.isupper()': False,
            'word_min_1.islower()': False,
            'word_min_1.istitle()': False,
#             'word_min_1.isdigit()': False,
            #'postag_min_1': '',
            
            'BOS': True if i == 0 else False,
            'EOS': True if i == len(sent)-1 else False,
        }
        
          
        if i < len(sent)-1:
            word_pls_1 = sent[i+1][1]
            #postag_pls_1 = sent[i+1][2]
            features.update({
#                 'word_pls_1.lower()': word_pls_1.lower(), 
                'word_pls_1': word_pls_1,
                'word_pls_1[-3:]': word_pls_1[-3:],
                'word_pls_1[-2:]': word_pls_1[-2:],
                'word_pls_1[-1:]': word_pls_1[-1:],
                'word_pls_1[:3]': word_pls_1[:3],
                'word_pls_1[:2]': word_pls_1[:2],
                'word_pls_1[:1]': word_pls_1[:1],
#                 'word_pls_1.isupper()': word_pls_1.isupper(),
                'word_pls_1.islower()': word_pls_1.islower(),
                'word_pls_1.istitle()': word_pls_1.istitle(),
#                 'word_pls_1.isdigit()': word_pls_1.isdigit(),
         #       'postag_pls_1': postag_pls_1
            })
        if i > 0:
            word_min_1 = sent[i-1][1]
            #postag_min_1 = sent[i-1][2]
            features.update({
#                 'word_min_1.lower()': word_min_1.lower(), 
                'word_min_1': word_min_1,
                'word_min_1[-3:]': word_min_1[-3:],
                'word_min_1[-2:]': word_min_1[-2:],
                'word_min_1[-1:]': word_min_1[-1:],
                'word_min_1[:3]': word_min_1[:3],
                'word_min_1[:2]': word_min_1[:2],
                'word_min_1[:1]': word_min_1[:1],
#                 'word_min_1.isupper()': word_min_1.isupper(),
                'word_min_1.islower()': word_min_1.islower(),
                'word_min_1.istitle()': word_min_1.istitle(),
#                 'word_min_1.isdigit()': word_min_1.isdigit(),
          #      'postag_min_1': postag_min_1
            })
        
        return features

def sent2features(sent):  
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for  token, label in sent]

# def sent2labels(sent):
#     return [label for title, token, postag, label, pred in sent]

#def sent2labelspred(sent):
 #   return [pred for title, token, postag, label, pred in sent]
    
def sent2tokens(sent):
    return [token for token, label in sent]

In [59]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
z = [sent2tokens(s) for s in sentences]

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
X_tr, X_te, y_tr, y_te = train_test_split(z, y, test_size=0.1, random_state=42)

In [62]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=True)

In [63]:
crf.fit(X_tr, y_tr)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [64]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [65]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-PER', 'I-PER']

In [66]:
y_pred = crf.predict(X_te)
metrics.flat_f1_score(y_te, y_pred,
                      average='weighted', labels=labels)

0.7632286309825603

In [67]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_te, y_pred, labels=sorted_labels, digits=2
))

             precision    recall  f1-score   support

      B-PER       0.94      0.81      0.87        90
      I-PER       0.79      0.65      0.72       201

avg / total       0.84      0.70      0.76       291



In [68]:
i = 13
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_te[i], y_te[i], y_pred[i]):
    if w != 0:
        print("{:15}: {:5} {}".format(w, t, pred))

Word           ||True ||Pred
dan            : O     O
demikianlah    : O     O
kami           : O     O
jadikan        : O     O
sebahagian     : O     O
orang-orang    : B-PER B-PER
yang           : I-PER I-PER
zalim          : I-PER I-PER
itu            : O     O
menjadi        : O     O
teman          : O     O
bagi           : O     O
sebahagian     : O     O
yang           : O     O
lain           : O     O
disebabkan     : O     O
apa            : O     O
yang           : O     O
mereka         : O     O
usahakan       : O     O


In [69]:
coba = crf.predict([['ibrahim']])
coba

[['B-PER']]

In [70]:
for i in range(90):
    print(X_te[i])

['bukanlah', 'kewajibanmu', 'menjadikan', 'mereka', 'mendapat', 'petunjuk', 'akan', 'tetapi', 'allah-lah', 'yang', 'memberi', 'petunjuk', 'memberi', 'taufiq', 'siapa', 'yang', 'dikehendaki-nya', 'dan', 'apa', 'saja', 'harta', 'yang', 'baik', 'yang', 'kamu', 'nafkahkan', 'di', 'jalan', 'allah', 'maka', 'pahalanya', 'itu', 'untuk', 'kamu', 'sendiri', 'dan', 'janganlah', 'kamu', 'membelanjakan', 'sesuatu', 'melainkan', 'karena', 'mencari', 'keridhaan', 'allah', 'dan', 'apa', 'saja', 'harta', 'yang', 'baik', 'yang', 'kamu', 'nafkahkan', 'niscaya', 'kamu', 'akan', 'diberi', 'pahalanya', 'dengan', 'cukup', 'sedang', 'kamu', 'sedikitpun', 'tidak', 'akan', 'dianiaya', 'dirugikan']
['dan', 'berimanlah', 'kamu', 'kepada', 'apa', 'yang', 'telah', 'aku', 'turunkan', 'al', 'quran', 'yang', 'membenarkan', 'apa', 'yang', 'ada', 'padamu', 'taurat', 'dan', 'janganlah', 'kamu', 'menjadi', 'orang', 'yang', 'pertama', 'kafir', 'kepadanya', 'dan', 'janganlah', 'kamu', 'menukarkan', 'ayat-ayat-ku', 'dengan'

['dan', 'apabila', 'datang', 'kepada', 'mereka', 'suatu', 'berita', 'tentang', 'keamanan', 'ataupun', 'ketakutan', 'mereka', 'lalu', 'menyiarkannya', 'dan', 'kalau', 'mereka', 'menyerahkannya', 'kepada', 'rasul', 'dan', 'ulil', 'amri', 'di', 'antara', 'mereka', 'tentulah', 'orang-orang', 'yang', 'ingin', 'mengetahui', 'kebenarannya', 'akan', 'dapat', 'mengetahuinya', 'dari', 'mereka', 'rasul', 'dan', 'ulil', 'amri', 'kalau', 'tidaklah', 'karena', 'karunia', 'dan', 'rahmat', 'allah', 'kepada', 'kamu', 'tentulah', 'kamu', 'mengikut', 'syaitan', 'kecuali', 'sebahagian', 'kecil', 'saja', 'di', 'antaramu']
['dan', 'sesungguhnya', 'jika', 'kamu', 'mendatangkan', 'kepada', 'orang-orang', 'yahudi', 'dan', 'nasrani', 'yang', 'diberi', 'al', 'kitab', 'taurat', 'dan', 'injil', 'semua', 'ayat', 'keterangan', 'mereka', 'tidak', 'akan', 'mengikuti', 'kiblatmu', 'dan', 'kamupun', 'tidak', 'akan', 'mengikuti', 'kiblat', 'mereka', 'dan', 'sebahagian', 'merekapun', 'tidak', 'akan', 'mengikuti', 'kiblat'

['hai', 'orang-orang', 'yang', 'beriman', 'jika', 'kamu', 'mentaati', 'orang-orang', 'yang', 'kafir', 'itu', 'niscaya', 'mereka', 'mengembalikan', 'kamu', 'ke', 'belakang', 'kepada', 'kekafiran', 'lalu', 'jadilah', 'kamu', 'orang-orang', 'yang', 'rugi']
['demikianlah', 'kisah', 'isa', 'kami', 'membacakannya', 'kepada', 'kamu', 'sebagian', 'dari', 'bukti-bukti', 'kerasulannya', 'dan', 'membacakan', 'al', 'quran', 'yang', 'penuh', 'hikmah']
['mereka', 'hendak', 'menipu', 'allah', 'dan', 'orang-orang', 'yang', 'beriman', 'padahal', 'mereka', 'hanya', 'menipu', 'dirinya', 'sendiri', 'sedang', 'mereka', 'tidak', 'sadar']
['apakah', 'kamu', 'mengira', 'bahwa', 'kamu', 'akan', 'masuk', 'surga', 'padahal', 'belum', 'datang', 'kepadamu', 'cobaan', 'sebagaimana', 'halnya', 'orang-orang', 'terdahulu', 'sebelum', 'kamu', 'mereka', 'ditimpa', 'oleh', 'malapetaka', 'dan', 'kesengsaraan', 'serta', 'digoncangkan', 'dengan', 'bermacam-macam', 'cobaan', 'sehingga', 'berkatalah', 'rasul', 'dan', 'orang-o

['allah', 'tidak', 'menghukum', 'kamu', 'disebabkan', 'sumpah-sumpahmu', 'yang', 'tidak', 'dimaksud', 'untuk', 'bersumpah', 'tetapi', 'dia', 'menghukum', 'kamu', 'disebabkan', 'sumpah-sumpah', 'yang', 'kamu', 'sengaja', 'maka', 'kaffarat', 'melanggar', 'sumpah', 'itu', 'ialah', 'memberi', 'makan', 'sepuluh', 'orang', 'miskin', 'yaitu', 'dari', 'makanan', 'yang', 'biasa', 'kamu', 'berikan', 'kepada', 'keluargamu', 'atau', 'memberi', 'pakaian', 'kepada', 'mereka', 'atau', 'memerdekakan', 'seorang', 'budak', 'barang', 'siapa', 'tidak', 'sanggup', 'melakukan', 'yang', 'demikian', 'maka', 'kaffaratnya', 'puasa', 'selama', 'tiga', 'hari', 'yang', 'demikian', 'itu', 'adalah', 'kaffarat', 'sumpah-sumpahmu', 'bila', 'kamu', 'bersumpah', 'dan', 'kamu', 'langgar', 'dan', 'jagalah', 'sumpahmu', 'demikianlah', 'allah', 'menerangkan', 'kepadamu', 'hukum-hukum-nya', 'agar', 'kamu', 'bersyukur', 'kepada-nya']
['hai', 'orang-orang', 'yang', 'beriman', 'janganlah', 'kamu', 'menanyakan', 'kepada', 'nabim

['kemudian', 'kamu', 'bani', 'israil', 'membunuh', 'dirimu', 'saudaramu', 'sebangsa', 'dan', 'mengusir', 'segolongan', 'daripada', 'kamu', 'dari', 'kampung', 'halamannya', 'kamu', 'bantu', 'membantu', 'terhadap', 'mereka', 'dengan', 'membuat', 'dosa', 'dan', 'permusuhan', 'tetapi', 'jika', 'mereka', 'datang', 'kepadamu', 'sebagai', 'tawanan', 'kamu', 'tebus', 'mereka', 'padahal', 'mengusir', 'mereka', 'itu', 'juga', 'terlarang', 'bagimu', 'apakah', 'kamu', 'beriman', 'kepada', 'sebahagian', 'al', 'kitab', 'taurat', 'dan', 'ingkar', 'terhadap', 'sebahagian', 'yang', 'lain?', 'tiadalah', 'balasan', 'bagi', 'orang', 'yang', 'berbuat', 'demikian', 'daripadamu', 'melainkan', 'kenistaan', 'dalam', 'kehidupan', 'dunia', 'dan', 'pada', 'hari', 'kiamat', 'mereka', 'dikembalikan', 'kepada', 'siksa', 'yang', 'sangat', 'berat', 'allah', 'tidak', 'lengah', 'dari', 'apa', 'yang', 'kamu', 'perbuat']
['kami', 'turunkan', 'al-quran', 'itu', 'agar', 'kamu', 'tidak', 'mengatakan', 'bahwa', 'kitab', 'itu'

In [71]:
for i in range(len(X_te)):
#     print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
    print(" ")
#     print(30 * "=")
    for w, t, pred in zip(X_te[i], y_te[i], y_pred[i]):
        if w != 0:
            print("{:15}: {:5} {}".format(w, t, pred))


 
bukanlah       : O     O
kewajibanmu    : O     O
menjadikan     : O     O
mereka         : O     O
mendapat       : O     O
petunjuk       : O     O
akan           : O     O
tetapi         : O     O
allah-lah      : O     O
yang           : O     O
memberi        : O     O
petunjuk       : O     O
memberi        : O     O
taufiq         : O     O
siapa          : O     O
yang           : O     O
dikehendaki-nya: O     O
dan            : O     O
apa            : O     O
saja           : O     O
harta          : O     O
yang           : O     O
baik           : O     O
yang           : O     O
kamu           : O     O
nafkahkan      : O     O
di             : O     O
jalan          : O     O
allah          : O     O
maka           : O     O
pahalanya      : O     O
itu            : O     O
untuk          : O     O
kamu           : O     O
sendiri        : O     O
dan            : O     O
janganlah      : O     O
kamu           : O     O
membelanjakan  : O     O
sesuatu        : O     

kerjakan       : O     O
 
dan            : O     O
demikianlah    : O     O
kami           : O     O
terangkan      : O     O
ayat-ayat      : O     O
al-quran       : O     O
supaya         : O     O
jelas          : O     O
jalan          : O     O
orang-orang    : B-PER B-PER
yang           : I-PER I-PER
saleh          : I-PER O
dan            : O     O
supaya         : O     O
jelas          : O     O
pula           : O     O
jalan          : O     O
orang-orang    : B-PER B-PER
yang           : I-PER I-PER
berdosa        : I-PER I-PER
 
maka           : O     O
berperanglah   : O     O
kamu           : O     O
pada           : O     O
jalan          : O     O
allah          : O     O
tidaklah       : O     O
kamu           : O     O
dibebani       : O     O
melainkan      : O     O
dengan         : O     O
kewajiban      : O     O
kamu           : O     O
sendiri        : O     O
kobarkanlah    : O     O
semangat       : O     O
para           : O     O
mukmin         : O     O
u

seorang        : O     O
mengingatkannya: O     O
janganlah      : O     O
saksi-saksi    : O     O
itu            : O     O
enggan         : O     O
memberi        : O     O
keterangan     : O     O
apabila        : O     O
mereka         : O     O
dipanggil      : O     O
dan            : O     O
janganlah      : O     O
kamu           : O     O
jemu           : O     O
menulis        : O     O
hutang         : O     O
itu            : O     O
baik           : O     O
kecil          : O     O
maupun         : O     O
besar          : O     O
sampai         : O     O
batas          : O     O
waktu          : O     O
membayarnya    : O     O
yang           : O     O
demikian       : O     O
itu            : O     O
lebih          : O     O
adil           : O     O
di             : O     O
sisi           : O     O
allah          : O     O
dan            : O     O
lebih          : O     O
menguatkan     : O     O
persaksian     : O     O
dan            : O     O
lebih          : O     O


tidak          : O     O
juga           : O     O
akan           : O     O
beriman        : O     O
kecuali        : O     O
jika           : O     O
allah          : O     O
menghendaki    : O     O
tetapi         : O     O
kebanyakan     : O     O
mereka         : O     O
tidak          : O     O
mengetahui     : O     O
 
ketika         : O     O
dua            : O     O
golongan       : O     O
dari           : O     O
padamu         : O     O
ingin          : O     O
mundur         : O     O
karena         : O     O
takut          : O     O
padahal        : O     O
allah          : O     O
adalah         : O     O
penolong       : O     O
bagi           : O     O
kedua          : O     O
golongan       : O     O
itu            : O     O
karena         : O     O
itu            : O     O
hendaklah      : O     O
kepada         : O     O
allah          : O     O
saja           : O     O
orang-orang    : B-PER B-PER
mukmin         : I-PER I-PER
bertawakkal    : I-PER I-PER
 
mereka   

apabila        : O     O
mereka         : O     O
telah          : O     O
pergi          : O     O
dari           : O     O
sisimu         : O     O
sebahagian     : O     O
dari           : O     O
mereka         : O     O
mengatur       : O     O
siasat         : O     O
di             : O     O
malam          : O     O
hari           : O     O
mengambil      : O     O
keputusan      : O     O
lain           : O     O
dari           : O     O
yang           : O     O
telah          : O     O
mereka         : O     O
katakan        : O     O
tadi           : O     O
allah          : O     O
menulis        : O     O
siasat         : O     O
yang           : O     O
mereka         : O     O
atur           : O     O
di             : O     O
malam          : O     O
hari           : O     O
itu            : O     O
maka           : O     O
berpalinglah   : O     O
kamu           : O     O
dari           : O     O
mereka         : O     O
dan            : O     O
tawakallah     : O     O


...

besertamu      : O     O
pemberi        : O     O
syafa'at       : O     O
yang           : O     O
kamu           : O     O
anggap         : O     O
bahwa          : O     O
mereka         : O     O
itu            : O     O
sekutu-sekutu  : O     O
tuhan          : O     O
di             : O     O
antara         : O     O
kamu           : O     O
sungguh        : O     O
telah          : O     O
terputuslah    : O     O
pertalian      : O     O
antara         : O     O
kamu           : O     O
dan            : O     O
telah          : O     O
lenyap         : O     O
daripada       : O     O
kamu           : O     O
apa            : O     O
yang           : O     O
dahulu         : O     O
kamu           : O     O
anggap         : O     O
sebagai        : O     O
sekutu         : O     O
allah          : O     O
 
allah          : O     O
tidak          : O     O
menghukum      : O     O
kamu           : O     O
disebabkan     : O     O
sumpah-sumpahmu: O     O
yang           : O     

menebus        : O     O
diri           : O     O
dengan         : O     O
emas           : O     O
yang           : O     O
sebanyak       : O     O
itu            : O     O
bagi           : O     O
mereka         : O     O
itulah         : O     O
siksa          : O     O
yang           : O     O
pedih          : O     O
dan            : O     O
sekali-kali    : O     O
mereka         : O     O
tidak          : O     O
memperoleh     : O     O
penolong       : O     O
 
orang-orang    : B-PER B-PER
kafir          : I-PER I-PER
itu            : O     O
membuat        : O     O
tipu           : O     O
daya           : O     O
dan            : O     O
allah          : O     O
membalas       : O     O
tipu           : O     O
daya           : O     O
mereka         : O     O
itu            : O     O
dan            : O     O
allah          : O     O
sebaik-baik    : O     O
pembalas       : O     O
tipu           : O     O
daya           : O     O
 
hai            : O     O
golongan     

bahwasanya     : O     O
laknat         : O     O
allah          : O     O
ditimpakan     : O     O
kepada         : O     O
mereka         : O     O
demikian       : O     O
pula           : O     O
laknat         : O     O
para           : O     O
malaikat       : O     O
dan            : O     O
manusia        : O     O
seluruhnya     : O     O
 
dan            : O     O
kaummu         : O     O
mendustakannya : O     O
azab           : O     O
padahal        : O     O
azab           : O     O
itu            : O     O
benar          : O     O
adanya         : O     O
katakanlah     : O     O
aku            : O     O
ini            : O     O
bukanlah       : O     O
orang          : B-PER O
yang           : I-PER O
diserahi       : I-PER O
mengurus       : I-PER O
urusanmu       : I-PER O
 
janganlah      : O     O
sekali-kali    : O     O
kamu           : O     O
menyangka      : O     O
hahwa          : O     O
orang-orang    : B-PER B-PER
yang           : I-PER I-PER
gembira      

fitnah         : O     O
syirik         : O     O
merekapun      : O     O
terjun         : O     O
kedalamnya     : O     O
karena         : O     O
itu            : O     O
jika           : O     O
mereka         : O     O
tidak          : O     O
membiarkan     : O     O
kamu           : O     O
dan            : O     O
tidak          : O     O
mau            : O     O
mengemukakan   : O     O
perdamaian     : O     O
kepadamu       : O     O
serta          : O     O
tidak          : O     O
menahan        : O     O
tangan         : O     O
mereka         : O     O
dari           : O     O
memerangimu    : O     O
maka           : O     O
tawanlah       : O     O
mereka         : O     O
dan            : O     O
bunuhlah       : O     O
mereka         : O     O
dan            : O     O
merekalah      : O     O
orang-orang    : O     B-PER
yang           : O     I-PER
kami           : O     O
berikan        : O     O
kepadamu       : O     O
alasan         : O     O
yang           : 

hal            : O     O
itu            : O     O
adalah         : O     O
lebih          : O     O
baik           : O     O
bagimu         : O     O
pada           : O     O
sisi           : O     O
tuhan          : O     O
yang           : O     O
menjadikan     : O     O
kamu           : O     O
maka           : O     O
allah          : O     O
akan           : O     O
menerima       : O     O
taubatmu       : O     O
sesungguhnya   : O     O
dialah         : O     O
yang           : O     O
maha           : O     O
penerima       : O     O
taubat         : O     O
lagi           : O     O
maha           : O     O
penyayang      : O     O
 
dan            : O     O
ingatlah       : O     O
ketika         : O     O
kami           : O     O
belah          : O     O
laut           : O     O
untukmu        : O     O
lalu           : O     O
kami           : O     O
selamatkan     : O     O
kamu           : O     O
dan            : O     O
kami           : O     O
tenggelamkan   : O     

In [72]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.5),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_tr, y_tr)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:   30.9s finished


Wall time: 32.4 s


In [73]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))


best params: {'c1': 0.3061716833797491, 'c2': 0.2941191348320434}
best CV score: 0.7321915094295345
model size: 0.01M


In [74]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [75]:
# _x = [s.parameters['c1'] for s in rs.grid_scores_]
# _y = [s.parameters['c2'] for s in rs.grid_scores_]
# _c = [s.mean_validation_score for s in rs.grid_scores_]

# fig = plt.figure()
# fig.set_size_inches(12, 12)
# ax = plt.gca()
# ax.set_yscale('log')
# ax.set_xscale('log')
# ax.set_xlabel('C1')
# ax.set_ylabel('C2')
# ax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(
#     min(_c), max(_c)
# ))

# ax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])

# print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))

In [76]:
rs.best_params_

{'c1': 0.3061716833797491, 'c2': 0.2941191348320434}

In [77]:
crf = rs.best_estimator_
y_pred = crf.predict(X_te)
print(metrics.flat_classification_report(
    y_te, y_pred, labels=sorted_labels, digits=2
))

             precision    recall  f1-score   support

      B-PER       0.95      0.81      0.87        90
      I-PER       0.80      0.65      0.72       201

avg / total       0.85      0.70      0.77       291



In [79]:
for i in range(len(X_te)):
#     print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
    print(" ")
#     print(30 * "=")
    for w, t, pred in zip(X_te[i], y_te[i], y_pred[i]):
        if w != 0:
            print("{:15}: {:5} {}".format(w, t, pred))

 
bukanlah       : O     O
kewajibanmu    : O     O
menjadikan     : O     O
mereka         : O     O
mendapat       : O     O
petunjuk       : O     O
akan           : O     O
tetapi         : O     O
allah-lah      : O     O
yang           : O     O
memberi        : O     O
petunjuk       : O     O
memberi        : O     O
taufiq         : O     O
siapa          : O     O
yang           : O     O
dikehendaki-nya: O     O
dan            : O     O
apa            : O     O
saja           : O     O
harta          : O     O
yang           : O     O
baik           : O     O
yang           : O     O
kamu           : O     O
nafkahkan      : O     O
di             : O     O
jalan          : O     O
allah          : O     O
maka           : O     O
pahalanya      : O     O
itu            : O     O
untuk          : O     O
kamu           : O     O
sendiri        : O     O
dan            : O     O
janganlah      : O     O
kamu           : O     O
membelanjakan  : O     O
sesuatu        : O     

kerjakan       : O     O
 
dan            : O     O
demikianlah    : O     O
kami           : O     O
terangkan      : O     O
ayat-ayat      : O     O
al-quran       : O     O
supaya         : O     O
jelas          : O     O
jalan          : O     O
orang-orang    : B-PER B-PER
yang           : I-PER I-PER
saleh          : I-PER O
dan            : O     O
supaya         : O     O
jelas          : O     O
pula           : O     O
jalan          : O     O
orang-orang    : B-PER B-PER
yang           : I-PER I-PER
berdosa        : I-PER I-PER
 
maka           : O     O
berperanglah   : O     O
kamu           : O     O
pada           : O     O
jalan          : O     O
allah          : O     O
tidaklah       : O     O
kamu           : O     O
dibebani       : O     O
melainkan      : O     O
dengan         : O     O
kewajiban      : O     O
kamu           : O     O
sendiri        : O     O
kobarkanlah    : O     O
semangat       : O     O
para           : O     O
mukmin         : O     O
u

seorang        : O     O
mengingatkannya: O     O
janganlah      : O     O
saksi-saksi    : O     O
itu            : O     O
enggan         : O     O
memberi        : O     O
keterangan     : O     O
apabila        : O     O
mereka         : O     O
dipanggil      : O     O
dan            : O     O
janganlah      : O     O
kamu           : O     O
jemu           : O     O
menulis        : O     O
hutang         : O     O
itu            : O     O
baik           : O     O
kecil          : O     O
maupun         : O     O
besar          : O     O
sampai         : O     O
batas          : O     O
waktu          : O     O
membayarnya    : O     O
yang           : O     O
demikian       : O     O
itu            : O     O
lebih          : O     O
adil           : O     O
di             : O     O
sisi           : O     O
allah          : O     O
dan            : O     O
lebih          : O     O
menguatkan     : O     O
persaksian     : O     O
dan            : O     O
lebih          : O     O


tidak          : O     O
juga           : O     O
akan           : O     O
beriman        : O     O
kecuali        : O     O
jika           : O     O
allah          : O     O
menghendaki    : O     O
tetapi         : O     O
kebanyakan     : O     O
mereka         : O     O
tidak          : O     O
mengetahui     : O     O
 
ketika         : O     O
dua            : O     O
golongan       : O     O
dari           : O     O
padamu         : O     O
ingin          : O     O
mundur         : O     O
karena         : O     O
takut          : O     O
padahal        : O     O
allah          : O     O
adalah         : O     O
penolong       : O     O
bagi           : O     O
kedua          : O     O
golongan       : O     O
itu            : O     O
karena         : O     O
itu            : O     O
hendaklah      : O     O
kepada         : O     O
allah          : O     O
saja           : O     O
orang-orang    : B-PER B-PER
mukmin         : I-PER I-PER
bertawakkal    : I-PER I-PER
 
mereka   

apabila        : O     O
mereka         : O     O
telah          : O     O
pergi          : O     O
dari           : O     O
sisimu         : O     O
sebahagian     : O     O
dari           : O     O
mereka         : O     O
mengatur       : O     O
siasat         : O     O
di             : O     O
malam          : O     O
hari           : O     O
mengambil      : O     O
keputusan      : O     O
lain           : O     O
dari           : O     O
yang           : O     O
telah          : O     O
mereka         : O     O
katakan        : O     O
tadi           : O     O
allah          : O     O
menulis        : O     O
siasat         : O     O
yang           : O     O
mereka         : O     O
atur           : O     O
di             : O     O
malam          : O     O
hari           : O     O
itu            : O     O
maka           : O     O
berpalinglah   : O     O
kamu           : O     O
dari           : O     O
mereka         : O     O
dan            : O     O
tawakallah     : O     O


...

besertamu      : O     O
pemberi        : O     O
syafa'at       : O     O
yang           : O     O
kamu           : O     O
anggap         : O     O
bahwa          : O     O
mereka         : O     O
itu            : O     O
sekutu-sekutu  : O     O
tuhan          : O     O
di             : O     O
antara         : O     O
kamu           : O     O
sungguh        : O     O
telah          : O     O
terputuslah    : O     O
pertalian      : O     O
antara         : O     O
kamu           : O     O
dan            : O     O
telah          : O     O
lenyap         : O     O
daripada       : O     O
kamu           : O     O
apa            : O     O
yang           : O     O
dahulu         : O     O
kamu           : O     O
anggap         : O     O
sebagai        : O     O
sekutu         : O     O
allah          : O     O
 
allah          : O     O
tidak          : O     O
menghukum      : O     O
kamu           : O     O
disebabkan     : O     O
sumpah-sumpahmu: O     O
yang           : O     

menebus        : O     O
diri           : O     O
dengan         : O     O
emas           : O     O
yang           : O     O
sebanyak       : O     O
itu            : O     O
bagi           : O     O
mereka         : O     O
itulah         : O     O
siksa          : O     O
yang           : O     O
pedih          : O     O
dan            : O     O
sekali-kali    : O     O
mereka         : O     O
tidak          : O     O
memperoleh     : O     O
penolong       : O     O
 
orang-orang    : B-PER B-PER
kafir          : I-PER I-PER
itu            : O     O
membuat        : O     O
tipu           : O     O
daya           : O     O
dan            : O     O
allah          : O     O
membalas       : O     O
tipu           : O     O
daya           : O     O
mereka         : O     O
itu            : O     O
dan            : O     O
allah          : O     O
sebaik-baik    : O     O
pembalas       : O     O
tipu           : O     O
daya           : O     O
 
hai            : O     O
golongan     

bahwasanya     : O     O
laknat         : O     O
allah          : O     O
ditimpakan     : O     O
kepada         : O     O
mereka         : O     O
demikian       : O     O
pula           : O     O
laknat         : O     O
para           : O     O
malaikat       : O     O
dan            : O     O
manusia        : O     O
seluruhnya     : O     O
 
dan            : O     O
kaummu         : O     O
mendustakannya : O     O
azab           : O     O
padahal        : O     O
azab           : O     O
itu            : O     O
benar          : O     O
adanya         : O     O
katakanlah     : O     O
aku            : O     O
ini            : O     O
bukanlah       : O     O
orang          : B-PER O
yang           : I-PER O
diserahi       : I-PER O
mengurus       : I-PER O
urusanmu       : I-PER O
 
janganlah      : O     O
sekali-kali    : O     O
kamu           : O     O
menyangka      : O     O
hahwa          : O     O
orang-orang    : B-PER B-PER
yang           : I-PER I-PER
gembira      

fitnah         : O     O
syirik         : O     O
merekapun      : O     O
terjun         : O     O
kedalamnya     : O     O
karena         : O     O
itu            : O     O
jika           : O     O
mereka         : O     O
tidak          : O     O
membiarkan     : O     O
kamu           : O     O
dan            : O     O
tidak          : O     O
mau            : O     O
mengemukakan   : O     O
perdamaian     : O     O
kepadamu       : O     O
serta          : O     O
tidak          : O     O
menahan        : O     O
tangan         : O     O
mereka         : O     O
dari           : O     O
memerangimu    : O     O
maka           : O     O
tawanlah       : O     O
mereka         : O     O
dan            : O     O
bunuhlah       : O     O
mereka         : O     O
dan            : O     O
merekalah      : O     O
orang-orang    : O     B-PER
yang           : O     I-PER
kami           : O     O
berikan        : O     O
kepadamu       : O     O
alasan         : O     O
yang           : 

hal            : O     O
itu            : O     O
adalah         : O     O
lebih          : O     O
baik           : O     O
bagimu         : O     O
pada           : O     O
sisi           : O     O
tuhan          : O     O
yang           : O     O
menjadikan     : O     O
kamu           : O     O
maka           : O     O
allah          : O     O
akan           : O     O
menerima       : O     O
taubatmu       : O     O
sesungguhnya   : O     O
dialah         : O     O
yang           : O     O
maha           : O     O
penerima       : O     O
taubat         : O     O
lagi           : O     O
maha           : O     O
penyayang      : O     O
 
dan            : O     O
ingatlah       : O     O
ketika         : O     O
kami           : O     O
belah          : O     O
laut           : O     O
untukmu        : O     O
lalu           : O     O
kami           : O     O
selamatkan     : O     O
kamu           : O     O
dan            : O     O
kami           : O     O
tenggelamkan   : O     

In [ ]:
y_te[0]